In [3]:
from ff_espn_api import League
import requests
from collections import defaultdict
import sys
import pandas as pd
import numpy as np

from urllib.request import urlopen 
from bs4 import BeautifulSoup

game_id_list = ['401012352']

game_id = '401012352'
url = 'http://site.api.espn.com/apis/site/v2/sports/football/college-football/summary?event='+game_id


r = requests.get(url, params='', cookies = '')
data = r.json()

In [340]:
print(data)

{'boxscore': {'teams': [{'team': {'id': '59', 'uid': 's:20~l:23~t:59', 'slug': 'georgia-tech-yellow-jackets', 'location': 'Georgia Tech', 'name': 'Yellow Jackets', 'abbreviation': 'GT', 'displayName': 'Georgia Tech Yellow Jackets', 'shortDisplayName': 'Yellow Jackets', 'color': '00223e', 'alternateColor': '002c56', 'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/59.png'}, 'statistics': [{'name': 'firstDowns', 'displayValue': '12', 'label': '1st Downs'}, {'name': 'thirdDownEff', 'displayValue': '3-13', 'label': '3rd down efficiency'}, {'name': 'fourthDownEff', 'displayValue': '3-6', 'label': '4th down efficiency'}, {'name': 'totalYards', 'displayValue': '219', 'label': 'Total Yards'}, {'name': 'netPassingYards', 'displayValue': '91', 'label': 'Passing'}, {'name': 'completionAttempts', 'displayValue': '5-14', 'label': 'Comp-Att'}, {'name': 'yardsPerPass', 'displayValue': '6.5', 'label': 'Yards per pass'}, {'name': 'rushingYards', 'displayValue': '128', 'label': 'Rushing'}, {'name': '

In [338]:
#view each section available in the json file
#for i in data:
#    print("HEADER",i)
#    for n in data[i]: 
#        print("SUB", n)

In [6]:
#Go through the sections 1 by 1 and grab the relevant information
#start with boxscore

#What data is available in boxscore?
#for i in data['boxscore']:
#    print("boxscore",i)
#Results: teams, players

#What data is available in teams section
#for i in data['boxscore']['teams']:
#    for key,value in i.items():
#        print("teams", key)
#Results teams,statistics

boxscore teams
boxscore players
teams team
teams statistics
teams team
teams statistics


In [7]:
#We'll start with team info in the teams section, lets grab each teams info, 
#viewing the data
for i in data['boxscore']['teams']:
    print(i['team'])
   

{'id': '59', 'uid': 's:20~l:23~t:59', 'slug': 'georgia-tech-yellow-jackets', 'location': 'Georgia Tech', 'name': 'Yellow Jackets', 'abbreviation': 'GT', 'displayName': 'Georgia Tech Yellow Jackets', 'shortDisplayName': 'Yellow Jackets', 'color': '00223e', 'alternateColor': '002c56', 'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/59.png'}
{'id': '61', 'uid': 's:20~l:23~t:61', 'slug': 'georgia-bulldogs', 'location': 'Georgia', 'name': 'Bulldogs', 'abbreviation': 'UGA', 'displayName': 'Georgia Bulldogs', 'shortDisplayName': 'Bulldogs', 'color': 'CC0000', 'alternateColor': '000000', 'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/61.png'}


In [90]:
#let's save this as a pandas dataframe

team0 = dict(data['boxscore']['teams'][0]['team'])
df0 = pd.DataFrame(list(team0.items())).T

team1 = dict(data['boxscore']['teams'][1]['team'])
df1 = pd.DataFrame(list(team1.items())).T

df_teams = pd.concat([df0.iloc[[1]], df1.iloc[[1]]])

#Grab the first row from df0 to set as column names
column_names = df0.iloc[[0]].values.tolist()
df_teams.columns = column_names

#
print(df_teams)


   id             uid                         slug      location  \
1  59  s:20~l:23~t:59  georgia-tech-yellow-jackets  Georgia Tech   
1  61  s:20~l:23~t:61             georgia-bulldogs       Georgia   

             name abbreviation                  displayName shortDisplayName  \
1  Yellow Jackets           GT  Georgia Tech Yellow Jackets   Yellow Jackets   
1        Bulldogs          UGA             Georgia Bulldogs         Bulldogs   

    color alternateColor                                               logo  
1  00223e         002c56  https://a.espncdn.com/i/teamlogos/ncaa/500/59.png  
1  CC0000         000000  https://a.espncdn.com/i/teamlogos/ncaa/500/61.png  


In [89]:
#Create an initial blank data frame
df_team_stats = pd.DataFrame(data=None)

for i in data['boxscore']['teams']:
    new_df = pd.DataFrame(i['statistics'])
    new_df = new_df.assign(team_id = i['team']['id'])
    new_df = new_df.assign(game_id = game_id)
    df_team_stats = pd.concat([df_team_stats,new_df])


#We can flip this to make it more compact
df_team_stats = df_team_stats.pivot_table(index = ['team_id','game_id'],columns='label', values='displayValue',  aggfunc='first').reset_index()

print ( df_team_stats)



label team_id    game_id 1st Downs 3rd down efficiency 4th down efficiency  \
0          59  401012352        12                3-13                 3-6   
1          61  401012352        25                 2-6                 2-2   

label Comp-Att Fumbles lost Interceptions thrown Passing Penalties Possession  \
0         5-14            0                    0      91      2-20      29:27   
1        15-19            1                    0     162      2-10      30:33   

label Rushing Rushing Attempts Total Yards Turnovers Yards per pass  \
0         128               46         219         0            6.5   
1         285               42         447         1            8.5   

label Yards per rush  
0                2.8  
1                6.8  


{'receiving', 'passing', 'kickReturns', 'puntReturns', 'punting', 'defensive', 'fumbles', 'kicking', 'rushing', 'interceptions'}


{'yo': 'a'}
4379396


In [100]:
data['boxscore']['teams']

[{'team': {'id': '59',
   'uid': 's:20~l:23~t:59',
   'slug': 'georgia-tech-yellow-jackets',
   'location': 'Georgia Tech',
   'name': 'Yellow Jackets',
   'abbreviation': 'GT',
   'displayName': 'Georgia Tech Yellow Jackets',
   'shortDisplayName': 'Yellow Jackets',
   'color': '00223e',
   'alternateColor': '002c56',
   'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/59.png'},
  'statistics': [{'name': 'firstDowns',
    'displayValue': '12',
    'label': '1st Downs'},
   {'name': 'thirdDownEff',
    'displayValue': '3-13',
    'label': '3rd down efficiency'},
   {'name': 'fourthDownEff',
    'displayValue': '3-6',
    'label': '4th down efficiency'},
   {'name': 'totalYards', 'displayValue': '219', 'label': 'Total Yards'},
   {'name': 'netPassingYards', 'displayValue': '91', 'label': 'Passing'},
   {'name': 'completionAttempts', 'displayValue': '5-14', 'label': 'Comp-Att'},
   {'name': 'yardsPerPass', 'displayValue': '6.5', 'label': 'Yards per pass'},
   {'name': 'rushingYards', 

In [38]:
#the next section in the call is boxscore players stats. First I want to develop a player database
athlete_list = []

for i in data['boxscore']['players']:
    for a in i['statistics']:
        for z in a['athletes']:
            #this is troubleshooting for erroneous ids which are negatives
            if(pd.to_numeric(z['athlete']['id'])) > 0:
                athlete_dict = dict()
                athlete_dict['id'] = z['athlete']['id']
                athlete_dict['name'] = z['athlete']['displayName']
                athlete_dict['guid'] = z['athlete']['guid']
                athlete_dict['link'] = z['athlete']['links'][0]['href']
                athlete_list.append(athlete_dict)

athlete_df = pd.DataFrame(athlete_list)

print(athlete_df[:5])

                               guid       id  \
0  d3095f49b0478d0dda8cb0dfba2ee5b1  3917810   
1  741da6a62d3356ebc470266aa1e7273c  4360578   
2  f46a3fce74c6755efb0364d2c2447423  4240482   
3  718c5787faececdbfd68870663fd0bc7  4240481   
4  d3095f49b0478d0dda8cb0dfba2ee5b1  3917810   

                                                link              name  
0  http://www.espn.com/college-football/player/_/...   TaQuon Marshall  
1  http://www.espn.com/college-football/player/_/...      James Graham  
2  http://www.espn.com/college-football/player/_/...     Tobias Oliver  
3  http://www.espn.com/college-football/player/_/...  Jerry Howard Jr.  
4  http://www.espn.com/college-football/player/_/...   TaQuon Marshall  


In [88]:
#the next section in the call is boxscore players stats, let's determine the possible sections of stats
stat_sections = set()

for i in data['boxscore']['players']:
    for a in i['statistics']:
        stat_sections.add(a['name'])
        
print(stat_sections)

{'receiving', 'passing', 'kickReturns', 'puntReturns', 'punting', 'defensive', 'fumbles', 'kicking', 'rushing', 'interceptions'}


In [94]:
#Creating a blank dictionary to house each of the stat dataframes we will create
df_game_player_stats = dict()

#using the stat_sections previously created, iterate and pull the stat data
for stat in stat_sections:
    #creating a blank list for the stat
    list_stat = []

    for i in data['boxscore']['players']:
            for a in i['statistics']:
                if a['name'] == stat :
                    #Creating the labels for the dataframe
                    keys_dict = (['player_id']+['team_id']+a['labels']+['stat_type'])
                    for athlete in a['athletes']:
                        #removing erronoeus data
                        if pd.to_numeric(athlete['athlete']['id']) >0:
                            #creating the values for the data frame
                            values_dict = ([athlete['athlete']['id']]+[i['team']['id']]+athlete['stats']+[stat])
                            #combining the stats into a list of dicts
                            list_stat.append(dict(zip(keys_dict,values_dict)))

    #transforming the list of dicts into a pd dataframe
    df_stat = pd.DataFrame(list_stat)
    
    #pushing each pd dataframe into the main stat dictionary
    df_game_player_stats[stat] = df_stat

#view a preview of each data set
for i in stat_sections:
      print(i,df_game_player_stats[i][:2])

receiving     AVG LONG REC TD YDS player_id  stat_type team_id
0  27.0   28   2  1  54   3917833  receiving      59
1  22.0   22   1  0  22   3917810  receiving      59
passing     AVG C/ATT INT   QBR TD YDS player_id stat_type team_id
0   5.8  4/12   0   4.7  1  69   3917810   passing      59
1  22.0   1/1   0  39.4  0  22   4360578   passing      59
kickReturns      AVG LONG NO TD  YDS player_id    stat_type team_id
0  100.0  100  1  1  100   4360590  kickReturns      59
1   16.0   16  1  0   16   4035005  kickReturns      61
puntReturns     AVG LONG NO TD YDS player_id    stat_type team_id
0  27.0   27  1  0  27   4035004  puntReturns      61
punting     AVG In 20 LONG NO TB  YDS player_id stat_type team_id
0  36.5     0   41  4  0  146   4240485   punting      59
1  45.0     0   45  1  0   45   4379396   punting      61
defensive   PD QB HUR SACKS SOLO TD  TFL TOT player_id  stat_type team_id
0  0      0     0    1  0    0   9   3116622  defensive      59
1  0      0     0    4  0 

In [ ]:
#A summary of what we have created so far
#1. Team level database of both teams involved
#2. Team game level stats
#3. A player database for all players mentioned in the box score
#4. Player/game level stat dataframes for each stat section (e.g. rushing, passing, etc.)

#still to do
#5. Game info (venue, attendance)
#6 Drive info (drive level data)
#7 Play by play info (play level data)


In [129]:
#5 Game info
print(data['gameInfo']['venue'])
        
venue_dict = dict()
venue_dict['id'] = data['gameInfo']['venue']['id']
venue_dict['fullName'] = data['gameInfo']['venue']['fullName']
venue_dict['city'] = data['gameInfo']['venue']['address']['city']
venue_dict['state'] = data['gameInfo']['venue']['address']['state']
venue_dict['zip'] = data['gameInfo']['venue']['address']['zipCode']
venue_dict['attendance'] = data['gameInfo']['attendance']

print(pd.DataFrame([venue_dict]))


{'id': '3917', 'fullName': 'Sanford Stadium', 'shortName': 'Sanford Stadium', 'address': {'city': 'Athens', 'state': 'GA', 'zipCode': '30602'}, 'capacity': 0, 'grass': False, 'images': [{'href': 'https://a.espncdn.com/i/venues/college-football/day/3917.jpg', 'width': 2000, 'height': 1125, 'alt': '', 'rel': ['full', 'day']}, {'href': 'https://a.espncdn.com/i/venues/college-football/day/interior/3917.jpg', 'width': 2000, 'height': 1125, 'alt': '', 'rel': ['full', 'day', 'interior']}]}
   attendance    city         fullName    id state    zip
0       92746  Athens  Sanford Stadium  3917    GA  30602


In [147]:
#6 Drive Data
# Step 1 will be to take a look at each section available

drive_sections = set()

for i in data['drives']['previous']:
    for a in i:
        drive_sections.add(a)
             
print(drive_sections)


{'isScore', 'result', 'displayResult', 'team', 'plays', 'start', 'yards', 'shortDisplayResult', 'offensivePlays', 'timeElapsed', 'id', 'description', 'end'}


In [214]:
#Create the drive dataframe

#Initiate a drive list
drive_list = []

#Loop through the drive data
for drive in data['drives']['previous']:
    drive_dict = dict()
    drive_dict['id'] = drive['id']
    drive_dict['plays'] = drive['description'].split()[0]
    drive_dict['yards'] = drive['description'].split()[2]
    drive_dict['time'] = drive['description'].split()[4]
    drive_dict['team'] = drive['team']['displayName']
    drive_dict['start_quarter'] = drive['start']['period']['number']
    drive_dict['start_clock'] = drive['start']['clock']['displayValue']
    drive_dict['start_yardline'] = drive['start']['yardLine']
    drive_dict['end_quarter'] = drive['end']['period']['number']
    #final plays of half do not have end clock
    try:
        drive_dict['end_clock'] = drive['end']['clock']['displayValue']
    except:
        drive_dict['end_clock'] = '0:00'
    drive_dict['end_yardline'] = drive['end']['yardLine']
    drive_dict['time_elapsed'] = drive['timeElapsed']['displayValue']
    drive_dict['yards'] = drive['yards']
    drive_dict['score'] = drive['isScore']
    drive_dict['off_play_ct'] = drive['offensivePlays']
    drive_dict['result'] = drive['result']    
    drive_list.append(drive_dict)

drive_df = pd.DataFrame(drive_list)

print(drive_df[:5])

  end_clock  end_quarter  end_yardline          id  off_play_ct plays result  \
0     10:23            1           100  4010123521            9     9     TD   
1      6:58            1            50  4010123522            6     6   PUNT   
2      0:51            1            97  4010123523           11    11     TD   
3      0:32            1             0  4010123524            0     0     TD   
4     12:56            2           100  4010123525            6     6     TD   

   score start_clock  start_quarter  start_yardline  \
0   True       15:00              1              25   
1  False       10:23              1              65   
2   True        6:58              1              12   
3   True        0:51              1               0   
4   True        0:32              1              37   

                          team  time time_elapsed  yards  
0             Georgia Bulldogs  4:37         4:37     75  
1  Georgia Tech Yellow Jackets  3:25         3:25     15  
2          

In [232]:
#7 play by play data

#initiate a blank list
play_list = []

#loop through each play
for drive in data['drives']['previous']:
    for play in drive['plays']:
        #initiate a dictionary
        play_dict = dict()
        play_dict['drive_id'] = drive['id']
        play_dict['play_id'] = play['id']
        play_dict['play_type_id'] = play['type']['id']
        play_dict['description'] = play['text']
        play_dict['awayScore'] = play['awayScore']
        play_dict['homeScore'] = play['homeScore']
        play_dict['quarter'] = play['period']['number']
        play_dict['clock'] = play['clock']['displayValue']
        play_dict['scoring_play'] = play['scoringPlay']
        play_dict['wallclock'] = play['wallclock']
        play_dict['start_down'] = play['start']['down']
        play_dict['start_distance'] = play['start']['distance']
        play_dict['start_yardline'] = play['start']['yardLine']
        play_dict['start_yards_to_ez'] = play['start']['yardsToEndzone']
        play_dict['start_team_id'] = play['start']['team']['id']
        play_dict['end_down'] = play['end']['down']
        play_dict['end_distance'] = play['end']['distance']
        play_dict['end_yardline'] = play['end']['yardLine']
        play_dict['end_yards_to_ez'] = play['end']['yardsToEndzone']
        play_dict['end_team_id'] = play['end']['team']['id']
        play_dict['stat_yards'] = play['statYardage']

        play_list.append(play_dict)

play_df = pd.DataFrame(play_list)

print(play_df[:1])
    

   awayScore  clock                                        description  \
0          0  15:00  Shawn Davis kickoff for 40 yds fair catch by W...   

     drive_id  end_distance  end_down end_team_id  end_yardline  \
0  4010123521            10         1          61            25   

   end_yards_to_ez  homeScore  ... play_type_id quarter  scoring_play  \
0               75          0  ...           53       1         False   

   start_distance  start_down  start_team_id start_yardline  \
0              10           1             59             65   

   start_yards_to_ez  stat_yards             wallclock  
0                 65           0  2018-11-24T17:02:03Z  

[1 rows x 21 columns]


In [251]:
#8 Prediction Data

predictor_list = []


ht = data['predictor']['homeTeam']
ht['home'] = True
predictor_list.append(ht)

at = data['predictor']['awayTeam']
at['home'] = False
predictor_list.append(at)


predictor_df = pd.DataFrame(predictor_list)
predictor_df

,gameProjection,home,id,teamChanceLoss
0,90.9,True,61,9.1
1,9.1,False,59,90.9


In [313]:
#pick data

pick_list = []

for i in data['pickcenter']:
    #print(i)
    pick_dict_away = dict()
    pick_dict_away['provider_id'] = i['provider']['id']
    pick_dict_away['provider_name'] = i['provider']['name']
    pick_dict_away['details'] = i['details']
    pick_dict_away['overUnder'] = i['overUnder']
    #spread
    if i['awayTeamOdds']['favorite'] == True:
        pick_dict_away['spread'] = (i['spread'])
    else:
        pick_dict_away['spread'] = -(i['spread'])
    pick_dict_away['favorite'] = i['awayTeamOdds']['favorite']
    #awayTeamOdds
    pick_dict_away['money_line'] = i['awayTeamOdds']['moneyLine']
    pick_dict_away['spread_odds'] = i['awayTeamOdds']['spreadOdds']
    pick_dict_away['team_ID'] = i['awayTeamOdds']['teamId']
    pick_dict_away['home_ind'] = 0
    #TRY CATCH
    try:
        pick_dict_away['avg_score'] = i['awayTeamOdds']['averageScore']
    except:
        pick_dict_away['avg_score'] = None
    try:
        pick_dict_away['win_pct'] = i['awayTeamOdds']['winPercentage']
    except:
        pick_dict_away['win_pct'] = None
    try:
        pick_dict_away['spread_return'] = i['awayTeamOdds']['spreadReturn']
    except:
         pick_dict_away['spread_return'] = None
    try:
         pick_dict_away['spread_wins'] = i['awayTeamOdds']['spreadRecord']['wins']
    except:
         pick_dict_away['spread_wins'] = None
    try:
         pick_dict_away['spread_losses'] = i['awayTeamOdds']['spreadRecord']['losses']
    except:
         pick_dict_away['spread_losses'] = None
    try:
         pick_dict_away['spread_pushes'] = i['awayTeamOdds']['spreadRecord']['pushes']
    except:
         pick_dict_away['spread_pushes'] = None
    pick_list.append(pick_dict_away)

for i in data['pickcenter']:
    #print(i)
    pick_dict_home = dict()
    pick_dict_home['provider_id'] = i['provider']['id']
    pick_dict_home['provider_name'] = i['provider']['name']
    pick_dict_home['details'] = i['details']
    pick_dict_home['overUnder'] = i['overUnder']
    #spread
    if i['homeTeamOdds']['favorite'] == True:
        pick_dict_home['spread'] = (i['spread'])
    else:
        pick_dict_home['spread'] = -(i['spread'])
    pick_dict_home['favorite'] = i['homeTeamOdds']['favorite']
    #homeTeamOdds
    pick_dict_home['money_line'] = i['homeTeamOdds']['moneyLine']
    pick_dict_home['spread_odds'] = i['homeTeamOdds']['spreadOdds']
    pick_dict_home['team_ID'] = i['homeTeamOdds']['teamId']
    pick_dict_home['home_ind'] = 1
    #TRY CATCH
    try:
        pick_dict_home['avg_score'] = i['homeTeamOdds']['averageScore']
    except:
        pick_dict_home['avg_score'] = None
    try:
        pick_dict_home['win_pct'] = i['homeTeamOdds']['winPercentage']
    except:
        pick_dict_home['win_pct'] = None
    try:
        pick_dict_home['spread_return'] = i['homeTeamOdds']['spreadReturn']
    except:
         pick_dict_home['spread_return'] = None
    try:
         pick_dict_home['spread_wins'] = i['homeTeamOdds']['spreadRecord']['wins']
    except:
         pick_dict_home['spread_wins'] = None
    try:
         pick_dict_home['spread_losses'] = i['homeTeamOdds']['spreadRecord']['losses']
    except:
         pick_dict_home['spread_losses'] = None
    try:
         pick_dict_home['spread_pushes'] = i['homeTeamOdds']['spreadRecord']['pushes']
    except:
         pick_dict_home['spread_pushes'] = None
    pick_list.append(pick_dict_home)
    
pick_df = pd.DataFrame(pick_list)
pick_df


,avg_score,details,favorite,home_ind,money_line,overUnder,provider_id,provider_name,spread,spread_losses,spread_odds,spread_pushes,spread_return,spread_wins,team_ID,win_pct
0,NaN,UGA -17.0,False,0,700,59.5,38,Caesars,17.0,NaN,-110.00,NaN,NaN,NaN,59,NaN
1,NaN,UGA -17.0,False,0,578,59.5,1004,consensus,17.0,NaN,-109.00,NaN,NaN,NaN,59,36.00
2,18.37,UGA -17.0,False,0,570,59.5,1003,numberfire,17.0,NaN,49.84,NaN,-0.048,NaN,59,10.33
3,21.30,UGA -17.0,False,0,560,60.0,1002,teamrankings,17.0,0.0,-111.00,0.0,NaN,0.0,59,NaN
4,NaN,UGA -17.0,True,1,-1100,59.5,38,Caesars,-17.0,NaN,-110.00,NaN,NaN,NaN,61,NaN
5,NaN,UGA -17.0,True,1,-861,59.5,1004,consensus,-17.0,NaN,-112.00,NaN,NaN,NaN,61,64.00
6,33.80,UGA -17.0,True,1,-825,59.5,1003,numberfire,-17.0,NaN,47.13,NaN,-0.100,NaN,61,89.67
7,37.90,UGA -17.0,True,1,-716,60.0,1002,teamrankings,-17.0,5.0,-102.00,0.0,NaN,8.0,61,NaN


In [269]:
#againstTheSpread - no data in here
#winprobability


win_prob_df = pd.DataFrame(data['winprobability'])
win_prob_df[:5]

,homeWinPercentage,playId,secondsLeft,tiePercentage
0,0.914,401012352101849902,3600,0.0
1,0.918,401012352101854401,3595,0.0
2,0.921,401012352101857901,3560,0.0
3,0.922,401012352101859901,3540,0.0
4,0.930,401012352101867401,3505,0.0


In [325]:
header_list = []

print(data['header']['week'])

for i in data['header']['competitions']:
    for n in i['competitors']:
        header_dict = dict()
        header_dict['conf_game'] = i['conferenceCompetition']
        header_dict['game_id'] = data['header']['id']
        header_dict['week'] = data['header']['week']
        header_dict['season'] = data['header']['season']['year']
        header_dict['id'] = n['id']
        header_dict['home_away'] = n['homeAway']
        header_dict['winner'] = n['winner']
        header_dict['score'] = n['score']
        header_dict['wins'] = n['record'][0]['displayValue'].split("-")[0]
        header_dict['losses'] = n['record'][0]['displayValue'].split("-")[1]
        try:
            header_dict['rank'] = n['rank']
        except:
            header_dict['rank'] = None
        header_list.append(header_dict)
        
header_df = pd.DataFrame(header_list)

header_df



13


,conf_game,game_id,home_away,id,losses,rank,score,season,week,winner,wins
0,False,401012352,home,61,1,5.0,45,2018,13,True,11
1,False,401012352,away,59,5,NaN,21,2018,13,False,7


['Atlantic Coast Conference ', ' Atlantic']
['SEC ', ' East']
{'fullViewLink': {'text': 'Full Standings', 'href': 'http://www.espn.com/college-football/standings'}, 'groups': [{'divisions': [{'header': 'Atlantic Coast Conference - Atlantic', 'standings': {'entries': [{'team': 'Boston College', 'link': 'http://www.espn.com/college-football/team/_/id/103/boston-college-eagles', 'id': '103', 'uid': 's:20~l:23~t:103', 'stats': [{'id': '0', 'name': 'overall', 'abbreviation': 'overall', 'displayName': 'Overall', 'shortDisplayName': 'OVER', 'description': 'Overall Record', 'type': 'total', 'summary': '1-0', 'displayValue': '1-0'}, {'id': '9', 'name': 'vsConf', 'abbreviation': 'CONF', 'displayName': 'vs. Conference', 'shortDisplayName': 'CONF', 'description': 'Conference Record', 'type': 'vsconf', 'summary': '1-0', 'displayValue': '1-0'}], 'logo': [{'href': 'https://a.espncdn.com/i/teamlogos/ncaa/500/103.png', 'width': 500, 'height': 500, 'alt': '', 'rel': ['full', 'default']}]}, {'team': 'Cle